## Imports
These experiments were run on Python 3.8. In the requirements.txt are the versions used for these packages.
- tqdm: For showing progress in loops.
- numpy and pandas: For data manipulation.
- cornac: For obtaining the recommendations.
- tensorflow: Required by cornac.
- torch: Required for the VAECF implementation of Cornac.

In [1]:
from datetime import datetime
from pathlib import Path
from logging import Formatter, StreamHandler, getLogger, INFO

from tqdm import tqdm
from cornac import Experiment
from cornac.eval_methods import RatioSplit, CrossValidation
from cornac.metrics import NDCG, Recall, Precision
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch
#from cornac.models import MF, WMF, SVD, VAECF
from cornac.models import VAECF, NeuMF, MCF, SVD, WMF, SKMeans, GMF, ItemKNN
from cornac.exception import ScoreException
from numpy import array, nan
from pandas import read_csv, DataFrame, Series

## Logger setup
Here we set up the logger for showing some info when executing this script.

In [2]:
logger = getLogger(__name__)
logger.setLevel(INFO)

ch = StreamHandler()
ch.setLevel(INFO)
ch.setFormatter(
    Formatter('%(asctime)s - %(levelname)s - %(message)s')
)

logger.addHandler(ch)

## Configuration variables
We define some variables used on the rest of the experiment.

### General config
Getting the date now and the name of the experiment.

In [3]:
now = f'{datetime.now():%Y%m%d%H%M}'
experiment_name = 'AMBAR'

### File and dir config
Getting the working directory with pathlib, and obtaining the csv to be used in cornac, and defining a results directory.

In [4]:
work_dir = Path('.').resolve()
data_file = work_dir / 'data' / experiment_name / 'ratings_info.csv'
results_dir = work_dir / 'results' / now

Here we make sure the results directory exists by creating it if it doesn't.

In [5]:
if not results_dir.exists():
    results_dir.mkdir(parents=True, exist_ok=True)

Also, we make sure the data file exists and is a file. Here we could also make sure that the file is an actual csv file.

In [6]:
if not data_file.exists() and data_file.is_file():
    print("Bad data file")

### Dataframe config
We define the names of the headers of each column to be identified by pandas. Also, we define the data type of the values in each cell of the user, item and rating. If the data has multiple data types, the val_dtype can be a list of type string compatible with pandas.

In [7]:
col_names = {
    'user': 'user_id',
    'item': 'track_id',
    'rating': 'rating'
}
val_dtype = 'int'

### Cornac config
Here we set up the k value, the test set size and the validation set size. Also we decide if we want to exclude unknown values or not.

In [8]:
k = 1000
test_size = 0.2
val_size = 0.1
exclude_unknown = True

## Function setup
We set up various utility functions to be used later. Mostly for exporting data and getting it in a format compatible with cornac.

set_data_to_tuple_list takes a dict of {user: [item_list, rating_list]}, process it and returns a tuple list of format [(user, item, rating)...].

In [9]:
def set_data_to_tuple_list(d: dict) -> list:
    result = []
    for user in d:
        transpose = array(d[user]).T
        for t in transpose:
            result.append((user,) + tuple(t))
    return result

list_to_dict converts a list into a dict using dict comprehension and enumerate.

In [10]:
def list_to_dict(l: list) -> dict:
    return {i: v for i, v in enumerate(l)}

get_set_dataframe process the raw data ({user: [item_list, rating_list]}), with the item ids and user ids, and converts it into a pandas DataFrame to be exported later.

In [11]:
# Transforma de formato ({user:[item_list, rating_list]})
# DataFrame final:
#    user_id  item_id  rating  item_idx  user_idx
# 0        0        0       5         0         0
# 1        0        1       3         1         0
# 2        1        1       4         1         1
# 3        1        2       2         2         1
def get_set_dataframe(set_data: dict, i_ids: list, u_ids: list) -> DataFrame:
    data_list = set_data_to_tuple_list(set_data)
    i_map = list_to_dict(i_ids)
    u_map = list_to_dict(u_ids)

    set_df = DataFrame(data_list,
                       columns=list(col_names.values()),
                       dtype=val_dtype)
    set_df['item_idx'] = set_df[col_names['item']]
    set_df['item'] = set_df[col_names['item']].replace(to_replace=i_map)
    set_df['user_idx'] = set_df[col_names['user']]
    set_df['user'] = set_df[col_names['user']].replace(to_replace=u_map)
    return set_df

In [12]:
logger.info('Experiment start...')
logger.info(f'{experiment_name}')
logger.info(f'{k=}')
logger.info(f'{work_dir=}')
logger.info(f'{data_file=}')
logger.info(f'{results_dir=}')

2025-01-13 00:11:47,055 - INFO - Experiment start...
2025-01-13 00:11:47,055 - INFO - AMBAR
2025-01-13 00:11:47,056 - INFO - k=1000
2025-01-13 00:11:47,057 - INFO - work_dir=WindowsPath('M:/Framework/AMBAR')
2025-01-13 00:11:47,057 - INFO - data_file=WindowsPath('M:/Framework/AMBAR/data/AMBAR/ratings_info.csv')
2025-01-13 00:11:47,058 - INFO - results_dir=WindowsPath('M:/Framework/AMBAR/results/202501130011')


Here we create the dataset out of the data file, the expected data is only with user, item and rating in that order. The name of the columns is defined in the set-up part, same with the data types.

For testing purposes before actually executing the full experiment, we left a filter that takes a sample of 50 users, and gets only the data of those 50 users. Please use it only to make sure that the script executes correctly from start to finish.

In [13]:
# user, item, rating
keys = ['0', '1', '2']

# Crea un diccionario que mapea cada clave a su tipo de dato
if isinstance(val_dtype, str):
    d_type = {key: val_dtype for key in keys}
elif isinstance(val_dtype, list):
    d_type = dict(zip(keys, val_dtype))
else:
    logger.error('Wrong type setup. Must be a type string or a list of type string.')
    exit()

logger.info('Loading data into triplets...')
df = read_csv(
    data_file,
    header=0,
    names=['0', '1', '2']
)[['0', '1', '2']].astype(d_type)

# FOR TESTING ONLY
# Selecciona aleatoriamente 50 usuarios unicos del dataframe
user_filter = Series(df['0'].unique()).sample(50, random_state= 123).to_list()
# Incluye solo filas donde estan los usuarios filtrados
df = df[df['0'].isin(user_filter)]
print(df.head)
data = list(df.to_records(index=False, column_dtypes=d_type))

2025-01-13 00:11:47,072 - INFO - Loading data into triplets...


<bound method NDFrame.head of                 0       1  2
1033    340282920    2632  2
1034    340282920    2634  2
1035    340282920    2635  2
1036    340282920    2636  2
1037    340282920    2643  2
...           ...     ... ..
105519  561593881   74572  1
105520  561593881  240714  1
105521  561593881   87783  1
105522  561593881  240716  1
105523  561593881   29142  1

[4147 rows x 3 columns]>


Here we create the Ratio Split that will be used by cornac. It splits the data into 3 sets randomly. 1 for test, 1 for train and 1 for validation.

In [14]:
logger.info('Creating ratio split...')
ratio_split = RatioSplit(
    data=data,
    test_size=test_size,
    val_size=val_size,
    exclude_unknowns=exclude_unknown,
    verbose=True,
    seed=123 # Añadida para poder reproducir resultados
)

cross_val = CrossValidation(
    data=data,
    n_folds=5,
    rating_threshold=1.0,
    verbose=True,
    seed=234
)

2025-01-13 00:11:47,134 - INFO - Creating ratio split...


rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 50
Number of items = 2258
Number of ratings = 2803
Max rating = 5.0
Min rating = 1.0
Global mean = 1.6
---
Test data:
Number of users = 50
Number of items = 2258
Number of ratings = 298
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 50
Number of items = 2258
Number of ratings = 147
---
Total users = 50
Total items = 2258
rating_threshold = 1.0
exclude_unknowns = True


c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 99 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 8 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 3 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


We define the metris here. In this experiment, we set up NDCG, Recall and Precision, using the k defined in the set-up.

In [15]:
metrics = [
    NDCG(k),
    Recall(k),
    Precision(k)
]

Also, we define the models with some previously obtained parameters. We could also define the hyperparameter calculation in this part, in this case, is important to leave a models variable with said configuration, so cornac can pick up the array and execute the calculation and exporting of the recommendations.

Because this script is assuming an array with models with parameters already predefined, in case of needing the best parameters obtained by cornac, the exporting of this must be done after running the experiment.

## Base models to compute

In [16]:
from cornac.models import BiVAECF
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch
original =  VAECF(
        name='originalvae',
        k=k,
        autoencoder_structure=[20],
        act_fn="tanh",
        likelihood="mult",
        n_epochs=100,
        batch_size=100,
        learning_rate=0.001,
        beta=1.0,
        use_gpu=True,
        verbose=True
    )

base_vaecf = VAECF(
    name='vaecf_default',
    k=k,
    autoencoder_structure=[20],
    act_fn="tanh",
    likelihood="mult",
    n_epochs=100,
    batch_size=100,
    learning_rate=0.001,
    beta=1.0,
    use_gpu=True,
        verbose=True)

base_vaecf_a32 = VAECF(
    name='vaecf_a32',
    k=k,
    autoencoder_structure=[32, 16],  # Deeper network for more complex patterns
    act_fn="relu",                   # ReLU often performs better in deep networks
    likelihood="mult",
    n_epochs=200,                    # More epochs for better convergence
    batch_size=256,                  # Larger batch size for faster training
    learning_rate=0.0005,           # Lower learning rate for stability
    beta=0.8,                       # Slightly lower beta to reduce KL impact
    use_gpu=True,
    verbose=True)

base_vaecf_a64 = VAECF(
    name='vaecf_a64',
    k=k,
    autoencoder_structure=[64],      # Wider single layer for more representation capacity
    act_fn="sigmoid",               # Sigmoid for smoother gradients
    likelihood="mult",
    n_epochs=150,                   # Balanced number of epochs
    batch_size=64,                  # Smaller batch size for better generalization
    learning_rate=0.002,           # Higher learning rate with sigmoid
    beta=1.2,                      # Higher beta for stronger regularization
    use_gpu=True,
    verbose=True)

# No usar, aun necesita testeo
base_mcf = MCF (
        k=k,
        max_iter= 100,
        learning_rate= 0.001,
        gamma = 0.9,
        lamda= 0.001,  
        verbose=True,
        #init_params= any
        #falta informacion del contexto
)

base_neumf = NeuMF (
    name= 'neumf_default',
    num_factors= 8,
    layers = (64, 32, 16, 8),
    act_fn = "relu",
    reg = 0,
    num_epochs= 20,
    batch_size = 256,
    num_neg = 4,
    lr = 0.001,
    learner = "adam",
    backend = "tensorflow",
    verbose = True
)

base_neumf_deep = NeuMF (
    name='neumf_deep',
    num_factors= 16,                    # Incrementado para capturar más factores latentes
    layers = (128, 64, 32, 16),        # Red más profunda para capturar patrones más complejos
    act_fn = "tanh",                   # Tanh para mejor gradiente en capas profundas
    reg = 0.01,                        # Regularización para evitar overfitting
    num_epochs= 50,                    # Más épocas para mejor convergencia
    batch_size = 128,                  # Batch size más pequeño para mejor generalización
    num_neg = 8,                       # Más muestras negativas para mejor discriminación
    lr = 0.0005,                       # Learning rate más bajo para estabilidad
    learner = "rmsprop",               # RMSprop para mejor manejo de gradientes
    backend = "tensorflow",
    verbose = True
)

# Experimento 3: Configuración con balance fairness-performance
base_neumf_fair = NeuMF (
    name='neumf_fair',
    num_factors= 32,                    # Mayor dimensionalidad para representación más rica
    layers = (256, 128, 64),           # Capas más anchas pero menos profundas
    act_fn = "elu",                    # ELU para mejor manejo de sesgos
    reg = 0.05,                        # Mayor regularización para reducir sesgos
    num_epochs= 30,                    # Balance entre entrenamiento y overfitting
    batch_size = 512,                  # Batch size grande para mejor estimación de gradientes
    num_neg = 6,                       # Balance en muestras negativas
    lr = 0.001,                        # Learning rate estándar
    learner = "adam",                  # Adam para adaptación automática
    backend = "tensorflow",
    verbose = True
)

base_svd =  SVD(
        max_iter=5,
        k=k,
        early_stop=True,
        verbose=True,
        lambda_reg=0.0001,
        learning_rate=0.0001
    )

base_skm = SKMeans (
    k = 5,
    max_iter= 100,
    tol=1e-6,
    verbose=True
    
)

base_gmf = GMF (
    num_factors=32,
    reg=0.1,
    num_epochs=50,
    batch_size=64,
    num_neg=8,
    lr=0.0005,
    learner='adagrad',
    verbose=True
)

itemknn = ItemKNN(
    k=50,
    similarity='cosine'
)
biVAECF = BiVAECF(
    name='vibae_exp1',
    k=k,
    encoder_structure=[20],
    act_fn="tanh",
    likelihood="pois",
    n_epochs=100,
    batch_size=100,
    learning_rate=0.001,
    beta_kl=1.0,
    use_gpu=True,
    verbose=True
)

gs_vae = GridSearch(
    model=biVAECF,
    space=[
        Discrete(name="k", values=[10,15,20,25]),
        Discrete(name="encoder_structure", values=[[20],]),
        Discrete(name="n_epochs", values=[100,150,200]),
        Discrete(name="batch_size", values=[100,150,200])
    ],
    metric=NDCG(1000),
    eval_method=ratio_split
)


In [17]:
# Modelos que le pasaremos a CORNAC para ejecutar los experimentos 
models = [
   #gs_vae
   # Define your model
   base_vaecf

]

In [18]:
# Obtener el total de usuarios del split de entrenamiento
total_users = ratio_split.train_set.num_users
# Obtener el total de items del split de entrenamiento
total_items = ratio_split.train_set.num_items
logger.info(f'{total_users=}')
logger.info(f'{total_items=}')

2025-01-13 00:11:47,244 - INFO - total_users=50
2025-01-13 00:11:47,245 - INFO - total_items=2258


After setting up the metrics and models, we export the test, train and validation data into the results directory.

In [19]:
logger.info('Exporting test data...')
get_set_dataframe(
    dict(ratio_split.test_set.user_data),
    list(ratio_split.test_set.item_ids),
    list(ratio_split.test_set.user_ids),
).to_csv(results_dir / 'test_set.csv')

2025-01-13 00:11:47,260 - INFO - Exporting test data...


In [20]:
logger.info('Exporting train data...')
get_set_dataframe(
    dict(ratio_split.train_set.user_data),
    list(ratio_split.train_set.item_ids),
    list(ratio_split.train_set.user_ids),
).to_csv(results_dir / 'train_set.csv')

2025-01-13 00:11:47,400 - INFO - Exporting train data...


# No comentar si se busca un set de validación

In [21]:
logger.info('Exporting validation data...')
get_set_dataframe(
    dict(ratio_split.val_set.user_data),
    list(ratio_split.val_set.item_ids),
    list(ratio_split.val_set.user_ids),
).to_csv(results_dir / 'val_set.csv')

2025-01-13 00:11:47,571 - INFO - Exporting validation data...


And we run the experiments with the defined variables.

In [22]:
logger.info('Running experiment...')
exp = Experiment(
    #eval_method=ratio_split,
    eval_method=cross_val,
    models=models,
    metrics=[NDCG(1000), Precision(1000), Recall(1000)],
    user_based=True,
)
exp.run()

#print(gs_vae.best_params)
#{'batch_size': 100, 'encoder_structure': [20], 'k': 15, 'n_epochs': 200}
train_data = ratio_split.train_set
train_results = models.evaluate(train_data, metrics)
print("Training Results:", train_results)

2025-01-13 00:11:47,681 - INFO - Running experiment...
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 116 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 6 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Fold: 1
---
Training data:
Number of users = 50
Number of items = 2501
Number of ratings = 3201
Max rating = 5.0
Min rating = 1.0
Global mean = 1.6
---
Test data:
Number of users = 50
Number of items = 2501
Number of ratings = 287
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 50
Number of items = 2501
Number of ratings = 287
---
Total users = 50
Total items = 2501

[vaecf_default] Training started!


100%|██████████| 100/100 [00:00<00:00, 120.12it/s, loss=9.93]



[vaecf_default] Evaluation started!


Ranking: 100%|██████████| 45/45 [00:00<00:00, 1046.45it/s]
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 109 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\Users\Juan Jose\AppData\Local\Programs\Python\Python38\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 6 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


...

100%|██████████| 100/100 [00:00<00:00, 125.00it/s, loss=9.98]



[vaecf_default] Evaluation started!


Ranking: 100%|██████████| 41/41 [00:00<00:00, 1078.92it/s]



TEST:
...
[vaecf_default]
       | NDCG@1000 | Train (s) | Test (s)
------ + --------- + --------- + --------
Fold 0 |    0.2047 |    3.3512 |   0.0460
Fold 1 |    0.1837 |    0.9520 |   0.0410
Fold 2 |    0.1834 |    0.8050 |   0.0410
Fold 3 |    0.1666 |    0.8430 |   0.0440
Fold 4 |    0.1825 |    0.8050 |   0.0420
------ + --------- + --------- + --------
Mean   |    0.1842 |    1.3512 |   0.0428
Std    |    0.0121 |    1.0014 |   0.0019



AttributeError: 'list' object has no attribute 'evaluate'

After running the experiment, we export the metrics obtained from the calculation into a csv using pandas.

In [ ]:
logger.info('Exporting metrics...')
metric_results = {
    exp.models[i].name: dict(exp.result[i].metric_avg_results)
    for i in range(len(models))
}
(DataFrame(metric_results)
 .reset_index()
 .rename(columns={'index': 'metric'})
 .to_csv(results_dir / 'metric_results.csv'))

And finally we export the recommendations. We use a custom multi loop to get the results.
- Here we first loop over the models of the experiment.
- We loop over the users map of cornac to get both the original id and the internal index of cornac.
- We get the scores for the users.
- We get the k top items using a combination of argsort and reversing of the list.
- We loop over the items map of cornac to get both the original id and the internal index of cornac.
- We get the score obtained from cornac, or nan in case of IndexError.
- We append the user and items, both the id and indexes, and the score to the result list.
- After all the loops are finished, we export the data into a csv file.

In [ ]:
logger.info('Processing models...')
for model in exp.models:
    model_result = []
    logger.info(f'Getting scores for {model.name}...')

    for user_id, user_index in tqdm(exp.eval_method.train_set.uid_map.items()):
        try:
            scores = model.score(user_index)
        except ScoreException:
            logger.error(f"{model.name}: Couldn't predict for user {user_index} ({user_id=})")
            continue

        top_items = list(reversed(scores.argsort()))[:k]

        for item_id, item_index in exp.eval_method.train_set.iid_map.items():
            if item_index not in top_items:
                continue

            try:
                score = scores[item_index]
            except IndexError:
                logger.error(
                    f"{model.name}: No score for item {item_index} ({item_id=}) in user {user_index} ({user_id=})"
                )
                score = nan

            model_result.append({
                'user_id': user_id,
                'user_index': user_index,
                'item_id': item_id,
                'item_index': item_index,
                'score': score
            })

    logger.info(f'Exporting {model.name}...')
    (DataFrame(model_result)
     .sort_values(by=['user_id', 'score'], ascending=[True, False])
     .to_csv(results_dir / f'{model.name}.csv'))

In [ ]:
print(gs_vae.score)